<a href="https://colab.research.google.com/github/MohamedHesham02/Sentiment-Analysis-Arabic-Tweets/blob/main/sentiment_analysis_model_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install colabcode
!pip install fastapi
!pip install joblib

In [3]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
import sklearn 
import requests, json
from nltk.tokenize import word_tokenize
import warnings
from colabcode import ColabCode
from fastapi import FastAPI
from nltk.stem.isri import ISRIStemmer
import pickle
import joblib
warnings.filterwarnings('ignore')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
url = 'https://recruitment.aimtechnologies.co/ai-tasks'
id = 1168104257578573824
id_formalization = [str(id)]
request_content = requests.post( url, data=json.dumps(id_formalization)).content
out = json.loads(request_content)
tweet = out[str(id)]
print(tweet)

ونسيتي انه من الجماعه الاحمديه او القاديانيه..واللي بيؤمنوا بوجود رسول بعد النبي محمد اسمه ميرزا غلام احمد.. https://t.co/PHJqDvKrER


In [5]:
# Arabic letters encodings

COMMA = u'\u060C'
SEMICOLON = u'\u061B'
QUESTION = u'\u061F'
HAMZA = u'\u0621'
ALEF_MADDA = u'\u0622'
ALEF_HAMZA_ABOVE = u'\u0623'
WAW_HAMZA = u'\u0624'
ALEF_HAMZA_BELOW = u'\u0625'
YEH_HAMZA = u'\u0626'
ALEF = u'\u0627'
TEH_MARBUTA = u'\u0629'
TATWEEL = u'\u0640'
LAM = u'\u0644'
HEH = u'\u0647'
YEH = u'\u064a'
ALEF_MAKSURA = u'\u0649'
HAMZA_ABOVE = u'\u0654'
HAMZA_BELOW = u'\u0655'
PERCENT = u'\u066a'
DECIMAL = u'\u066b'
THOUSANDS = u'\u066c'
STAR = u'\u066d'
FULL_STOP = u'\u06d4'
BYTE_ORDER_MARK = u'\ufeff'
MULITIPLICATION_SIGN = u'\u00D7'
DIVISION_SIGN = u'\u00F7'


# Tashkeel
FATHATAN = u'\u064b'
DAMMATAN = u'\u064c'
KASRATAN = u'\u064d'
FATHA = u'\u064e'
DAMMA = u'\u064f'
KASRA = u'\u0650'
SHADDA = u'\u0651'
SUKUN = u'\u0652'

#Ligatures
LAM_ALEF = u'\ufefb'
LAM_ALEF_HAMZA_ABOVE = u'\ufef7'
LAM_ALEF_HAMZA_BELOW = u'\ufef9'
LAM_ALEF_MADDA_ABOVE = u'\ufef5'

HARAKAT_PAT = re.compile(u"["+u"".join([FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SUKUN, SHADDA])+u"]")
HAMZAT_PAT = re.compile(u"["+u"".join([WAW_HAMZA, YEH_HAMZA])+u"]")
ALEFAT_PAT = re.compile(u"["+u"".join([ALEF_MADDA, ALEF_HAMZA_ABOVE, ALEF_HAMZA_BELOW, HAMZA_ABOVE, HAMZA_BELOW])+u"]")
LAMALEFAT_PAT = re.compile(u"["+u"".join([LAM_ALEF, LAM_ALEF_HAMZA_ABOVE, LAM_ALEF_HAMZA_BELOW, LAM_ALEF_MADDA_ABOVE])+u"]")

WESTERN_ARABIC_NUMERALS = ['0','1','2','3','4','5','6','7','8','9']
EASTERN_ARABIC_NUMERALS = [u'\u06F0', u'\u06F1', u'\u06F2', u'\u06F3', u'\u0664', u'\u06F5', u'\u0666', u'\u06F7', u'\u06F8', u'\u06F9']

eastern_to_western_numerals = {}
for i in range(len(EASTERN_ARABIC_NUMERALS)):
    eastern_to_western_numerals[EASTERN_ARABIC_NUMERALS[i]] = WESTERN_ARABIC_NUMERALS[i]

arabic_punctuations = COMMA + SEMICOLON + QUESTION + PERCENT + DECIMAL + THOUSANDS + STAR + FULL_STOP + MULITIPLICATION_SIGN + DIVISION_SIGN
all_punctuations = string.punctuation + arabic_punctuations + '()[]{}' # English & Arabic punctuations

all_punctuations = ''.join(list(set(all_punctuations)))

In [6]:
def strip_tashkeel(text):
    text = HARAKAT_PAT.sub('', text)
    return text 

def strip_tatweel(text):
    return re.sub(u'[%s]' % TATWEEL, '', text)

def remove_non_arabic(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", text,  flags=re.UNICODE).split())

def keep_arabic_english_n_symbols(text):
    return ' '.join(re.sub(u"[^\u0621-\u063A\u0640-\u064aa-zA-Z#@_:/ ]", "", text,  flags=re.UNICODE).split())

def normalize_hamza(text):
    text = ALEFAT_PAT.sub(ALEF, text)
    text = HAMZAT_PAT.sub(HAMZA, text)
    return text

def normalize_lamalef(text): 
    text = LAMALEFAT_PAT.sub((LAM, ALEF), text)
    return text

def normalize_spellerrors(text):
    text = re.sub(u'[%s]' % TEH_MARBUTA, HEH, text)
    return re.sub(u'[%s]' % ALEF_MAKSURA, YEH, text)

def remove_underscore(text):
    return ' '.join(text.split('_'))

def remove_retweet_tag(text):
    tag = re.compile('rt @[a-zA-Z0-9_]+:|@[a-zA-Z0-9_]+')
    text = tag.sub('', text)
    hashtag = re.compile('\#')
    text = hashtag.sub('', text)
    return text

def replace_emails(text):
    emails = re.findall(r'[\w\.-]+@[\w\.-]+', text)
    for email in emails:
        text = text.replace(email,'#')
    return text

def replace_urls(text):
    return re.sub(r"http\S+|www.\S+", "#", text)

def convert_eastern_to_western_numerals(text):
    for num in EASTERN_ARABIC_NUMERALS:
        text = text.replace(num, eastern_to_western_numerals[num])
    return text

def remove_all_punctuations(text):
    for punctuation in all_punctuations:
        text = text.replace(punctuation, ' ')
    return text

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags 
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub('', text)
    return text

def replace_phone_numbers(text):
    return re.sub(r'\d{10}', '#', text)

def remove_extra_spaces(text):
    return ' '.join(text.split())

def normalize_tweet(text):
    new_text = text.lower()
    new_text = normalize_hamza(new_text)
    new_text = strip_tashkeel(new_text)
    new_text = strip_tatweel(new_text)
    new_text = normalize_spellerrors(new_text)
    new_text = remove_retweet_tag(new_text)
    new_text = replace_emails(new_text)
    new_text = remove_underscore(new_text)
    new_text = replace_phone_numbers(new_text)
    new_text = remove_all_punctuations(new_text)
    new_text = replace_urls(new_text)
    new_text = convert_eastern_to_western_numerals(new_text)
    new_text = keep_arabic_english_n_symbols(new_text)
    new_text = remove_non_arabic(new_text)
    new_text = remove_extra_spaces(new_text)
    
    return new_text

In [7]:
tweet = normalize_tweet(tweet)

In [8]:
stop = stopwords.words('arabic')
arab_stopwords = set(nltk.corpus.stopwords.words("arabic"))
arab_stopwords = '|'.join(arab_stopwords)
tweet=tweet.replace(arab_stopwords, '')

token = word_tokenize(tweet)
string = []
st = ISRIStemmer()

for i in token:
    stemmed = st.stem(i)
    string.append(stemmed)
    tweet = ' '.join(string)
print(tweet)

vectorizer = joblib.load("/content/vectorizer_feature.pkl")
vector = vectorizer.transform([tweet])

print(vector)

ونس انه من جمع حمد او قاديانيه ولل يءم وجد رسل بعد نبي حمد اسم يرز غلم حمد
  (0, 9584)	0.30171005846322607
  (0, 9178)	0.39023936849983903
  (0, 9008)	0.22261082392275838
  (0, 8828)	0.14859920879248614
  (0, 8473)	0.21509687543275444
  (0, 7481)	0.2708471910193225
  (0, 7242)	0.10657057577183937
  (0, 4690)	0.39023936849983903
  (0, 2781)	0.2537080171110839
  (0, 1971)	0.40772083226500533
  (0, 1592)	0.17868393822023684
  (0, 793)	0.16055050772703905
  (0, 495)	0.18738566198075468
  (0, 478)	0.17855710484790785
  (0, 239)	0.2155773103457617


In [9]:
app = FastAPI(title="ML Models as API on Google Colab", description="with FastAPI and ColabCode", version="1.0")

model = joblib.load('/content/finalized_model.sav')

@app.post("/api", tags=["prediction"])
async def get_predictions(tweet:str):
  
  tweet = normalize_tweet(tweet)
  stop = stopwords.words('arabic')
  arab_stopwords = set(nltk.corpus.stopwords.words("arabic"))
  arab_stopwords = '|'.join(arab_stopwords)
  tweet=tweet.replace(arab_stopwords, '')

  token = word_tokenize(tweet)
  string = []
  st = ISRIStemmer()

  for i in token:
      stemmed = st.stem(i)
      string.append(stemmed)
      tweet = ' '.join(string)

  vectorizer = joblib.load("/content/vectorizer_feature.pkl")
  vector = vectorizer.transform([tweet])

  dialects = {
      0: 'SA',
      1: 'DZ',
      2: 'AE',
      3: 'BH',
      4: 'JO',
      5: 'YE',
      6: 'OM',
      7: 'MA',
      8: 'LY',
      9: 'IQ',
      10: 'PL',
      11: 'LB',
      12: 'SY',
      13: 'SD',
      14: 'EG',
      15: 'QA',
      16: 'TN',
      17: 'KW'
  }

  prediction = model.predict(vector)[0]
  dialect_class = dialects[prediction]
  return "This tweet is for class: ", dialect_class

In [ ]:
cc = ColabCode(port=12000, code=False)
cc.run_app(app=app)

INFO:     Started server process [1615]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://1597-35-231-207-149.ngrok.io" -> "http://localhost:12000"
INFO:     45.243.211.219:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     45.243.211.219:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     45.243.211.219:0 - "GET /docs HTTP/1.1" 200 OK
